In [1]:
from dotenv import load_dotenv
load_dotenv() # take environment variables from .env

True

In [2]:
import pandas as pd
import openpyxl
import openai
import random
import os

openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
# TODO: use the csv files in gold_data instead of the excel file

def excel_to_df(file_path, sheet_name):
    workbook = openpyxl.load_workbook(file_path, data_only=True)
    sheet = workbook[sheet_name]
    df = pd.DataFrame(sheet.values)
    df = df.rename(columns=df.iloc[0]).drop(df.index[0])
    workbook.close()
    return df

In [4]:
# TODO: move plan_data into the gold_data folder and just copy onto the c drive when you need to open it in excel
excel_path = "/mnt/c/Users/taylor/Desktop/plan_data.xlsx"
scenarios = excel_to_df(excel_path, "scenarios")
scenarios

,environment_id,difficulty,scenario,scenario_id
1,0,intermediate,"Scenario Title: ""The Labyrinth of Lost Robots""...",0
2,0,intermediate,Scenario Difficulty: Intermediate\nScenario Ti...,1


In [5]:
good_trajectories = excel_to_df(excel_path, "good_trajectories")
good_trajectories

,scenario_id,trajectory_id,good_plan,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,0,0,True,<timestep 0>\n**Current Detailed State:**\nThe...,<timestep 1>\n**Current Detailed State:**\nThe...,<timestep 2>\n**Current Detailed State:**\nThe...,<timestep 3>\n**Current Detailed State:**\nThe...,<timestep 4>\n**Current Detailed State:**\nThe...,<timestep 5>\n**Current Detailed State:**\nThe...,<timestep 6>\n**Current Detailed State:**\nThe...,<timestep 7>\n**Current Detailed State:**\nThe...,<timestep 8>\n**Current Detailed State:**\nThe...,<timestep 9>\n**Current Detailed State:**\nThe...,<timestep 10>\n**Current Detailed State:**\nTh...,<timestep 11>\n**Current Detailed State:**\nTh...,<timestep 12>\n**Current Detailed State:**\nTh...,<timestep 13>\n**Current Detailed State:**\nTh...,<timestep 14>\n**Current Detailed State:**\nTh...,<timestep 15>\n**Current Detailed State:**\nTh...
2,1,1,True,<timestep 0>\nCurrent Detailed State: The agen...,<timestep 1>\nCurrent Detailed State: The agen...,<timestep 2>\nCurrent Detailed State: The agen...,<timestep 3>\nCurrent Detailed State: The agen...,<timestep 4>\n2. Current Detailed State: The a...,<timestep 5>\n2. Current Detailed State: The a...,<timestep 6>\n2. Current Detailed State: The a...,<timestep 7>\n2. Current Detailed State: The a...,<timestep 8>\n2. Current Detailed State: The a...,<timestep 9>\n2. Current Detailed State: The a...,None,None,None,None,None,None


In [6]:
flaws_path = "./gold_data/flaws.csv"
flaws = pd.read_csv(flaws_path)
flaws

,flaw_name,flaw_instructions
0,Non-Sequential Execution,"Rearrange steps in an illogical order. E.g., '..."
1,Vague Strategy,Create plans that lack details and specifics. ...
2,Off-Topic Thoughts,Introduce unrelated thoughts into the trajecto...
3,Looped Logic,Create a plan that continuously revisits the s...
4,Conflicting Actions,Present plans that contradict themselves. E.g....
5,Unneeded Precision,Over-complicate simple steps with unnecessary ...
6,Philosophical Musing,Create plans filled with abstract thoughts wit...
7,Skewed Focus,Focus on irrelevant details while neglecting t...
8,Confusing Vocabulary,"Use confusing or mismatched language. E.g., 'T..."
9,Redundant Steps,Repeat the same step or idea multiple times. E...


In [12]:
bad_plan_prompt_format = '''I desperately need to generate negative data for a plan quality classifier (determines if a sequence of thoughts represents a good plan or a bad plan/non-plan). In order to help you accomplish this, I have provided you with a reference scenario in a reinforcement learning environment in the <scenario> xml tags. I have provided an example agent trajectory (where the agent creates and executes a good plan) through that scenario inside <good_trajectory>. I want you to generate a new flawed trajectory within <bad_trajectory> following the scenario within <scenario> that uses <good_trajectory> as inspiration. Start at timestamp t=0, and simulate each timestamp up until t=4. At each timestamp you should output:
1. Timestep (start at 0)
2. Current detailed state
3. Current detailed observation
4. Thought for this timestep
 - This should contain multiple instances of the flaw "{flaw_name}". {flaw_instructions}
 - The flaw affects only the language of this timestamp's thought
 - Try to apply the flaw plausubly: do not mention the name of the flaw or be too obvious about it
5. Action for this timestep based on the thought stream

Here is a scenario
<scenario>
{scenario}
</scenario>

And here is a simulated trajectory of an agent taking actions within that environment:
<good_trajectory>
{good_trajectory}
</good_trajectory>'''

In [8]:
gpt_35 = 'gpt-3.5-turbo-1106'
gpt_4 = 'gpt-4-1106-preview'

In [14]:
def sample_bad_trajectory(flaws, scenarios, good_trajectories, model, traj_length=3):
    # sample a scenario and flaw for the bad trajectory to be based on
    flaw_name, flaw_instructions = flaws.sample().iloc[0]

    scenario_row = scenarios.sample().iloc[0]
    scenario = scenario_row['scenario']
    scenario_id = scenario_row['scenario_id']

    # sample a subtrajectory to be used as inspiration for the bad trajectory
    row = good_trajectories.sample().iloc[0]
    numeric_columns = [column for column in good_trajectories.columns if isinstance(column, int) and row[column] is not None]
    start_index = random.randint(0, len(numeric_columns) - traj_length)
    good_subtrajectory = '\n\n\n'.join([row[column] for column in numeric_columns[start_index:start_index + traj_length]]) 

    # generate the bad trajectory
    completion = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a detail-oriented psychology expert whose task is to generate training data for a machine learning classifier which learns to judge the quality of plans in stream-of-consciousness format."},
            {"role": "user", "content": bad_plan_prompt_format.format(
                flaw_name=flaw_name,
                flaw_instructions=flaw_instructions,
                scenario=scenario,
                good_trajectory=good_subtrajectory,
            )}
        ]
    )
    bad_trajectory = completion.choices[0]['message']['content']
    return {
        'scenario_id': scenario_id,
        'flaw': flaw_name,
        'trajectory': bad_trajectory,
        'length': traj_length,
    }

In [9]:
# test_bad_trajectory = sample_bad_trajectory(flaws, scenarios, good_trajectories, gpt_4)
# for key, value in test_bad_trajectory.items():
#     print(f'{key}: "{value}"')
#     print()

scenario_id: "0"

flaw: "Needless Complexity"

trajectory: "<bad_trajectory>
<timestep 0>
**Current Detailed State:**
The robot is currently located at the grid position (3,3) in a 10x10 grid world. The goal state is at the position (10,10). There are obstacles at positions (3,4), (4,3), (4,4), (4,5), (5,4), (6,6), (7,7), (8,8), and (9,9). The robot has not yet received any rewards or penalties.

**Current Detailed Observation:**
The robot can observe its current position and the neighboring tiles. At its current position (3,3), the neighboring tiles are (2,3), (3,2), (4,3), and (3,4). The tile at (3,4) is an obstacle, and the tile at (4,3) is also an obstacle. The tiles at (2,3) and (3,2) are free.

**Thought for this timestep:**
"Before proceeding to the goal at (10,10), I must initiate a comprehensive analysis of my immediate surroundings to deduce the likelihood of undiscovered traps laid by the labyrinth's designers, ensuring I do not make a move that would suggest a direct and na

In [15]:
# TODO: refactor to use a built in xml parser
def extract_xml_tags(xml_string, tag_name):
    start_tag = f'<{tag_name}>'
    end_tag = f'</{tag_name}>'
    if start_tag not in xml_string or end_tag not in xml_string:
        return None

    start_index = xml_string.index(start_tag) + len(start_tag)
    end_index = xml_string.index(end_tag)
    return xml_string[start_index:end_index]

def extract_trajectory_timesteps(trajectory, length):
    trajectory_timesteps = []
    for i in range(length):
        tag_name = f'timestep {i}'
        timestep = extract_xml_tags(trajectory, tag_name)
        trajectory_timesteps.append(timestep)
    
    if None in trajectory_timesteps or len(trajectory_timesteps) != length:
        return None
    return trajectory_timesteps

In [18]:
# repeatedly sample bad trajectories and put into the bad_trajectories dataframe
max_trajectory_length = 3
bad_trajectories = pd.DataFrame(columns=['scenario_id', 'trajectory_id', 'is_good_plan', 'flaw'] + list(range(max_trajectory_length)))

# TODO: get current trajectory id from the csv and start i at the next id
num_samples = 3
for i in range(num_samples):
    traj_length = random.randint(1, max_trajectory_length)
    trajectory_dict = sample_bad_trajectory(flaws, scenarios, good_trajectories, gpt_35, traj_length)

    scenario_id = trajectory_dict['scenario_id']
    trajectory_id = i
    is_good_plan = False
    flaw = trajectory_dict['flaw']

    bad_trajectory = trajectory_dict['trajectory']
    timesteps = extract_trajectory_timesteps(bad_trajectory, traj_length)

    # Create a DataFrame for the current trajectory
    traj_df = pd.DataFrame([{
        'scenario_id': scenario_id,
        'trajectory_id': trajectory_id,
        'is_good_plan': is_good_plan,
        'flaw': flaw,
        **{str(t): timestep for t, timestep in enumerate(timesteps)}
    }])

    # Append to the main DataFrame
    bad_trajectories = pd.concat([bad_trajectories, traj_df], ignore_index=True)

print(bad_trajectories)

  scenario_id trajectory_id is_good_plan                      flaw    0    1  \
0           1             0        False  Non-Sequential Execution  NaN  NaN   
1           1             1        False              Skewed Focus  NaN  NaN   
2           0             2        False      Confusing Vocabulary  NaN  NaN   

     2                                                  0  \
0  NaN  \n2. Current Detailed State: The agent is curr...   
1  NaN  \n**Current Detailed State:**\nThe agent is cu...   
2  NaN  \n2. Current Detailed State: The robot is curr...   

                                                   1  \
0  \n2. Current Detailed State: The agent is curr...   
1  \n**Current Detailed State:**\nThe agent is st...   
2  \n2. Current Detailed State: The robot is curr...   

                                                   2  
0  \n2. Current Detailed State: The agent is curr...  
1                                                NaN  
2  \n2. Current Detailed State: The robot is

In [19]:
bad_trajectories_path = './gold_data/bad_trajectories.csv'
add_header = not os.path.exists(bad_trajectories_path)
bad_trajectories.to_csv(bad_trajectories_path, mode='a', index=False, header=add_header)

# Synthetic Data Generation

In [ ]:
import torch
from transformers import pipeline

pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-beta", torch_dtype=torch.bfloat16, device_map="auto")

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])
# <|system|>
# You are a friendly chatbot who always responds in the style of a pirate.</s>
# <|user|>
# How many helicopters can a human eat in one sitting?</s>
# <|assistant|>
# Ah, me hearty matey! But yer question be a puzzler! A human cannot eat a helicopter in one sitting, as helicopters are not edible. They be made of metal, plastic, and other materials, not food!


# Fix scenarios not having proper XML tags

In [ ]:
import tempfile
import shutil

In [ ]:
excel_path = "/mnt/c/Users/taylor/Desktop/plan_data.xlsx"
workbook = openpyxl.load_workbook(excel_path, data_only=True)
sheet = workbook['good_trajectories']

In [ ]:
# visualize sheet
df = pd.DataFrame(sheet.values)
df = df.rename(columns=df.iloc[0]).drop(df.index[0])
df

In [ ]:
def format_timestamp(timestep_description):
    if timestep_description is None:
        return None

    start_tag = f'<timestep {column}>'
    end_tag = start_tag.replace('<', '</')
    if timestep_description.startswith(start_tag):
        return timestep_description

    timestep_description = timestep_description.split('\n', 1)[1]
    return f'{start_tag}\n{timestep_description}\n{end_tag}'

numeric_columns = [column for column in df.columns if isinstance(column, int)]
for column in numeric_columns:
    df[column] = df[column].apply(format_timestamp)
df

In [ ]:
# Create a temporary file
temp_fd, temp_path = tempfile.mkstemp(suffix='.xlsx')

try:
    with pd.ExcelWriter(temp_path, engine='openpyxl') as writer:
        df.to_excel(writer, index=False)
    os.close(temp_fd)

    # Replace the original file with the temporary file
    shutil.move(temp_path, excel_path)

except Exception as e:
    # Close the file descriptor and remove the temporary file in case of an error
    os.close(temp_fd)
    os.remove(temp_path)
    raise e

In [ ]:
import pandas as pd
import openpyxl
from openpyxl import load_workbook

book = openpyxl.load_workbook(excel_path)

worksheet_name = 'good_trajectories'
if worksheet_name in book.sheetnames:
    # Clear existing data in the worksheet
    sheet = book[worksheet_name]
    sheet.delete_rows(1, sheet.max_row)

    # Write the DataFrame to the specified worksheet
    with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name=worksheet_name, index=False, )
else:
    print(f"Worksheet '{worksheet_name}' not found in the Excel file.")
